# Amazon Rekognition Custom Labels

First, you'll need to install the AWS SDK for Python-- [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html). This module generates our service clients and simplifies accessing AWS APIs.

In [ ]:
!pip3 install boto3

Next, configure this notebooks global settings. 

1. Set the `bucket_name` variable to an S3 bucket within your AWS account
1. Set the `region_name` to the S3 bucket's region
1. Create the `rekognition` client for the same region

In [ ]:
import boto3
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont

In [ ]:
bucket_name = 'cv-on-aws-book-xxxx'
region_name = 'us-east-2'
object_name = 'chapter_03/locate_packt_logo.jpeg'
rekognition = boto3.client('rekognition', region_name=region_name)

Copy images to S3 folder.

In [ ]:
!aws s3 sync 03_RekognitionCustomLabels/images s3://cv-on-aws-book-xxxx/chapter_03/images --region us-east-2

After training the model, follow the steps below.

Now, let's invoke the `DetectCustomLabels` API to inspect the `chapter_03/locate_packt_logo.jpeg` image.

In [ ]:
model='arn:aws:rekognition:us-east-2:264701418850:project/Packt-logo-detection/version/Packt-logo-detection.2022-08-23T20.23.19/1661300599617'

min_confidence=85

In [ ]:
response = rekognition.detect_custom_labels(
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': object_name
        }
    },
    MinConfidence=min_confidence,
    ProjectVersionArn=model
)

print(response)

Let's draw bounding box around the label.

In [ ]:
# Load image from S3 bucket
s3_connection = boto3.resource('s3')

s3_object = s3_connection.Object(bucket_name,object_name)
s3_response = s3_object.get()

stream = io.BytesIO(s3_response['Body'].read())
image=Image.open(stream)

# Ready image to draw bounding boxes on it.
imgWidth, imgHeight = image.size
draw = ImageDraw.Draw(image)

# calculate and display bounding boxes for each detected custom label
print('Detected custom labels for ' + object_name)
for customLabel in response['CustomLabels']:
    print('Label \"' + str(customLabel['Name'])+'\"')
    print('Confidence ' + str(customLabel['Confidence']))
    if 'Geometry' in customLabel:
        box = customLabel['Geometry']['BoundingBox']
        left = imgWidth * box['Left']
        top = imgHeight * box['Top']
        width = imgWidth * box['Width']
        height = imgHeight * box['Height']

        fnt = ImageFont.load_default()
        draw.text((left,top), customLabel['Name'], fill='#00d400', font=fnt)

        print('Left: ' + '{0:.0f}'.format(left))
        print('Top: ' + '{0:.0f}'.format(top))
        print('Label Width: ' + "{0:.0f}".format(width))
        print('Label Height: ' + "{0:.0f}".format(height))

        points = (
            (left,top),
            (left + width, top),
            (left + width, top + height),
            (left , top + height),
            (left, top))
        draw.line(points, fill='#00d400', width=5)

image.show()